# Week 1 - Exercice 2

1. À votre retour, codez le modèle Pydantic correspondant à cette dernière version de votre structure de sortie technique.
2. Créez la chaîne LCEL simple (Prompt adapté -> Modèle Ollama -> PydanticOutputParser avec votre nouveau modèle).
3. Invoquez-la pour tester.

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pprint import pprint

In [2]:
import sys

sys.path.append("../")

from agents.technical_analyst.technical_analysis_pydantic_model import TickerTechnicalAnalysis, ShortTimeframeData, LongTimeframeData

In [3]:
MODEL = "cogito:8b"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"
invocation = {"stock": "APPLE"}

In [4]:
model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 2, num_predict=1000 * 2)

In [5]:
system_template = ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                #    "Remove the text between the '<think>' and '</think>' tags from the output."
                )

In [6]:
short_technical_parser = PydanticOutputParser(pydantic_object=ShortTimeframeData)

short_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock"],
    partial_variables={"format_instructions": short_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

In [7]:
prompt = short_prompt_template.invoke(invocation)
prompt

ChatPromptValue(messages=[SystemMessage(content='As trading technical analyst expert, your task is to generate the technical analysis report for APPLE at the specified JSON format. Use the data (simulated for the exercice) to generate the technical analysis of APPLE action totally filling the JSON schema described below. The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"BollignerBandsRawValues": {"description": "Bollinger Bands raw values provided by the Bollinger Bands tool.

In [8]:
response = model.invoke(prompt)

In [9]:
response

AIMessage(content='{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "supports_evaluation": "TESTING_SUPPORT",\n        "support_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "support_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_support_level": "1,750.00",\n        "middle_support_level": "1,600.00",\n        "far_support_level": "1,450.00",\n        "raw_tool_data": {\n            "close_support_value": 1750,\n            "middle_support_value": 1600,\n            "far_support_value": 1450\n        }\n    },\n    "resistances_evaluation": {\n        "resistances_evaluation": "CONSOLIDATING_NEAR_SUPPORT_RESISTANCE",\n        "resistance_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "resistance_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_resistance_level": "1,800.00",\n        "middle_resistance_level": "1,700.00",\n        "far_resistance_level": "1,650.00",\n        "raw_tool_data": {\n            "close_resistance_valu

In [10]:
response.response_metadata

{'model': 'cogito:8b',
 'created_at': '2025-05-18T11:40:36.0489128Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 21908703500,
 'load_duration': 21025600,
 'prompt_eval_count': 5462,
 'prompt_eval_duration': 9885300,
 'eval_count': 756,
 'eval_duration': 21872047400,
 'model_name': 'cogito:8b'}

In [11]:
response.usage_metadata

{'input_tokens': 5462, 'output_tokens': 756, 'total_tokens': 6218}

In [12]:
response.content

'{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "supports_evaluation": "TESTING_SUPPORT",\n        "support_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "support_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_support_level": "1,750.00",\n        "middle_support_level": "1,600.00",\n        "far_support_level": "1,450.00",\n        "raw_tool_data": {\n            "close_support_value": 1750,\n            "middle_support_value": 1600,\n            "far_support_value": 1450\n        }\n    },\n    "resistances_evaluation": {\n        "resistances_evaluation": "CONSOLIDATING_NEAR_SUPPORT_RESISTANCE",\n        "resistance_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "resistance_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_resistance_level": "1,800.00",\n        "middle_resistance_level": "1,700.00",\n        "far_resistance_level": "1,650.00",\n        "raw_tool_data": {\n            "close_resistance_value": 1800,\n       

In [13]:
tmp = response.content.split("\n\n")
tmp

['{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "supports_evaluation": "TESTING_SUPPORT",\n        "support_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "support_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_support_level": "1,750.00",\n        "middle_support_level": "1,600.00",\n        "far_support_level": "1,450.00",\n        "raw_tool_data": {\n            "close_support_value": 1750,\n            "middle_support_value": 1600,\n            "far_support_value": 1450\n        }\n    },\n    "resistances_evaluation": {\n        "resistances_evaluation": "CONSOLIDATING_NEAR_SUPPORT_RESISTANCE",\n        "resistance_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "resistance_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_resistance_level": "1,800.00",\n        "middle_resistance_level": "1,700.00",\n        "far_resistance_level": "1,650.00",\n        "raw_tool_data": {\n            "close_resistance_value": 1800,\n      

In [14]:
len(tmp)

1

In [21]:
tmp[-1]

'{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "$ref": "#/$defs/SupportEvaluation"\n    },\n    "resistances_evaluation": {\n        "$ref": "#/$defs/ResistanceEvaluation",\n        "raw_tool_data": {\n            "close_resistance_value": 145.00,\n            "middle_resistance_value": 148.50,\n            "far_resistance_value": 152.75\n        }\n    },\n    "prices_evaluation": {\n        "$ref": "#/$defs/PricesEvaluation"\n    },\n    "indicators": {\n        "$ref": "#/$defs/Indicators",\n        "rsi_evaluation": null,\n        "macd_evaluation": {\n            "primary_macd_trend": "STRONG_BULLISH",\n            "primary_macd_trend_number_of_touches": 12\n        }\n    },\n    "volumes_evaluation": {\n        "$ref": "#/$defs/VolumesEvaluation"\n    },\n    "short_timeframe_data_synthesis": {\n        "$ref": "#/$defs/Synthesis",\n        "conclusion": "Strong bullish momentum continues with positive RSI and MACD signals.",\n        "synthese_remarkable_va

In [16]:
import json
import ast
import re

In [17]:
tmp2 = re.sub(r"(?<=\d),(?=\d)", "", tmp[-1])
# tmp2 = re.sub(r"null", "None", tmp2)
tmp2

'{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "$ref": "#/$defs/SupportEvaluation"\n    },\n    "resistances_evaluation": {\n        "$ref": "#/$defs/ResistanceEvaluation",\n        "raw_tool_data": {\n            "close_resistance_value": 156.78,\n            "middle_resistance_value": 159.23,\n            "far_resistance_value": 162.45\n        }\n    },\n    "prices_evaluation": {\n        "$ref": "#/$defs/PricesEvaluation"\n    },\n    "indicators": {\n        "$ref": "#/$defs/Indicators",\n        "rsi_evaluation": null,\n        "macd_evaluation": null,\n        "bollinger_bands_evaluation": null\n    },\n    "volumes_evaluation": {\n        "$ref": "#/$defs/VolumesEvaluation"\n    },\n    "short_timeframe_data_synthesis": {\n        "$ref": "#/$defs/Synthesis",\n        "conclusion": "Apple is in a strong upward trend, with technical indicators confirming potential buying opportunities.",\n        "synthese_trading_action": "BUY",\n        "synthese_support_

In [18]:
# ast.literal_eval(tmp2)

In [15]:
short_technical_parser.invoke(tmp[-1])

ShortTimeframeData(data_timeframe=5, supports_evaluation=SupportEvaluation(evaluation='TESTING_SUPPORT', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.TAKE_PROFIT_ZONE: 'TAKE_PROFIT_ZONE'>, close_level='1,750.00', middle_level='1,600.00', far_level='1,450.00', raw_tool_data=SupportRawToolData(close_value=1750.0, middle_value=1600.0, far_value=1450.0)), resistances_evaluation=ResistanceEvaluation(evaluation='CONSOLIDATING_NEAR_SUPPORT_RESISTANCE', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.TAKE_PROFIT_ZONE: 'TAKE_PROFIT_ZONE'>, close_level='1,800.00', middle_level='1,700.00', far_level='1,650.00', raw_tool_data=ResistanceRawToolData(close_value=1800.0, middle_value=1700.0, far_value=1650.0)), prices_evaluation=PricesEvaluation(trend='STRONG_BULLISH',

In [ ]:


short_chain = short_prompt_template | model | short_technical_parser
short_json = short_chain.invoke(invocation)
short_json

KeyboardInterrupt: 

In [18]:
pprint(short_json.model_dump_json())

('{"data_timeframe":5,"supports_evaluation":{"evaluation":"Strong support '
 'levels identified at $130 and '
 '$135","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"TAKE_PROFIT_ZONE","close_level":"$130","middle_level":"$132.5","far_level":"$135","raw_tool_data":{"close_value":130.0,"middle_value":132.5,"far_value":135.0}},"resistances_evaluation":{"evaluation":"Key '
 'resistance levels at $165 and '
 '$170","interaction_status":"PRICE_BELOW_RESISTANCE","interaction_implication":"STOP_LOSS_ZONE","close_level":"$165","middle_level":"$167.5","far_level":"$170","raw_tool_data":{"close_value":165.0,"middle_value":167.5,"far_value":170.0}},"prices_evaluation":{"trend":"BULLISH","trading_action":"BUY","primary_trend":"STRONG_BULLISH","primary_trend_number_of_touches":3,"secondary_trend":"BULLISH","secondary_trend_number_of_touches":2,"minor_trend":"CONSOLIDATION","minor_trend_number_of_touches":1,"raw_tool_data":{"prices_value":155.0},"trend_evaluation":"BULLISH","cha

In [7]:
long_technical_parser = PydanticOutputParser(pydantic_object=LongTimeframeData)

long_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock"],
    partial_variables={"format_instructions": long_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

long_chain = long_prompt_template | model | long_technical_parser
long_json = long_chain.invoke(invocation)
long_json

LongTimeframeData(data_timeframe=60, supports_evaluation=SupportEvaluation(evaluation='Strong Support Levels Present', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='144.50', middle_level='139.25', far_level='134.00', raw_tool_data=SupportRawToolData(close_value=144.5, middle_value=139.25, far_value=134.0)), resistances_evaluation=ResistanceEvaluation(evaluation='Multiple Resistance Levels Active', interaction_status=<SupportResistanceInteractionStatus.PRICE_BELOW_RESISTANCE: 'PRICE_BELOW_RESISTANCE'>, interaction_implication=<SupportResistanceInteractionImplication.STOP_LOSS_ZONE: 'STOP_LOSS_ZONE'>, close_level='155.75', middle_level='150.50', far_level='145.25', raw_tool_data=ResistanceRawToolData(close_value=155.75, middle_value=150.5, far_value=145.25)), prices_evaluation=PricesEvaluation(trend='BULLISH', tradi

In [19]:
pprint(long_json.model_dump_json())

('{"data_timeframe":60,"supports_evaluation":{"evaluation":"Strong Support '
 'Levels '
 'Present","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"POTENTIAL_BUY_ZONE","close_level":"144.50","middle_level":"139.25","far_level":"134.00","raw_tool_data":{"close_value":144.5,"middle_value":139.25,"far_value":134.0}},"resistances_evaluation":{"evaluation":"Multiple '
 'Resistance Levels '
 'Active","interaction_status":"PRICE_BELOW_RESISTANCE","interaction_implication":"STOP_LOSS_ZONE","close_level":"155.75","middle_level":"150.50","far_level":"145.25","raw_tool_data":{"close_value":155.75,"middle_value":150.5,"far_value":145.25}},"prices_evaluation":{"trend":"BULLISH","trading_action":"OUTPERFORM","primary_trend":"STRONG_BULLISH","primary_trend_number_of_touches":5,"secondary_trend":"BEARISH","secondary_trend_number_of_touches":2,"minor_trend":"CONSOLIDATION","minor_trend_number_of_touches":1,"raw_tool_data":{"prices_value":149.5},"trend_evaluation":"Positive '
 'Mome

In [8]:
global_system_template = ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "The JSON of the short timeframe analysis is {short_json}."
                   "The JSON of the long timeframe analysis is {long_json}."
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

In [9]:
global_invocation = {"stock": "APPLE", "short_json":short_json.model_dump_json(), "long_json":long_json.model_dump_json()}

In [11]:
global_technical_parser = PydanticOutputParser(pydantic_object=TickerTechnicalAnalysis)

global_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=global_system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "short_json", "long_json"],
    partial_variables={"format_instructions": global_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

global_chain = global_prompt_template | model | global_technical_parser
global_json = global_chain.invoke(global_invocation)
global_json

TickerTechnicalAnalysis(name_of_the_company='Apple Inc.', isin_of_the_company='US0378331005', time_of_the_report=datetime.datetime(2023, 8, 16, 10, 0, tzinfo=TzInfo(UTC)), synthesis=Synthesis(conclusion='Strong bullish trend with multiple support levels at $130-$135 and resistance zones at $165-$170. Key technical patterns include a Double Bottom Pattern forming on the short timeframe, while a Bullish Flag Pattern is developing on the long timeframe.', synthese_remarkable_values=None, synthese_trading_action=<TradingActions.BUY: 'BUY'>, synthese_support_resistance_comment='Key support levels at $130-$135 and resistance zones at $165-$170. Multiple touches confirmed on support levels with strong volume confirmation.', synthese_support_resistance_interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, synthese_support_resistance_interaction_implication=<SupportResistanceInteractionImplication.TAKE_PROFIT_ZONE: 'TAKE_PROFIT_ZONE'>))

In [ ]:
pprint(global_json.model_dump_json())

('{"name_of_the_company":"Apple '
 'Inc.","isin_of_the_company":"US0378331005","time_of_the_report":"2023-08-16T10:00:00Z","synthesis":{"conclusion":"Strong '
 'bullish trend with multiple support levels at $130-$135 and resistance zones '
 'at $165-$170. Key technical patterns include a Double Bottom Pattern forming '
 'on the short timeframe, while a Bullish Flag Pattern is developing on the '
 'long '
 'timeframe.","synthese_remarkable_values":null,"synthese_trading_action":"BUY","synthese_support_resistance_comment":"Key '
 'support levels at $130-$135 and resistance zones at $165-$170. Multiple '
 'touches confirmed on support levels with strong volume '
 'confirmation.","synthese_support_resistance_interaction_status":"PRICE_ABOVE_SUPPORT","synthese_support_resistance_interaction_implication":"TAKE_PROFIT_ZONE"}}')
